In [2]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.layers import LSTM, TimeDistributed
# from keras.layers import Concatenate, Flatten
# from keras.layers import GRU, Conv2D, MaxPooling2D
# from keras.layers import Input, Reshape, Dot, Add
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.optimizers import RMSprop
# # from keras.utils.vis_utils import plot_model
# import keras
# import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
# import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
# tf.set_random_seed(seed_val)

In [3]:
def parseVec(file, delimiter):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    vocab_size  = min(len(word2int), vocab_size)
    embeddings = np.zeros((vocab_size + 1, embed_size), dtype=np.float64)
    for i in range(1, vocab_size):
        try:
            line = lines[i][:-1].split(delimiter)
            word = line[0]
            if 'unk' in word:
                continue
            if word in word2int:
                wordvec = np.array([np.float64(j) for j in line[1:] if j != ''])
                embeddings[word2int[word]] = wordvec
        except Exception as e:
            print(lines[i])
            print(e)
    return embeddings

In [6]:
for dataId in [20]:
    words = read_file(filename='data/corpus/{0}'.format(dataId))
    xvocab, xword2int, xint2word = build_vocab(words)
    words, word2freq = min_count_threshold(words)
    vocab, word2int, int2word = build_vocab(words)

    for trainId in range(2):
        emb = parseVec('results/arabic/{0}_100-{1}.vec'.format(dataId, trainId), ' ')
        nr = evaluate(word2int, normalize(emb), embed_size=emb.shape[1], corpus='data/corpus/{0}'.format(dataId), analogy='data/ar_questions.txt')
        ur = evaluate(word2int, emb, embed_size=emb.shape[1], corpus='data/corpus/{0}'.format(dataId), analogy='data/ar_questions.txt')
        nsemantic = 0
        nsynatctic = 0
        for nrr in nr:
            if 'total' in nrr:
                continue
            if "gram" in nrr:
                nsynatctic += nr[nrr]/6
            else:
                nsemantic += nr[nrr]/9

        usemantic = 0
        usynatctic = 0
        for nrr in ur:
            if 'total' in nrr:
                continue
            if "gram" in nrr:
                usemantic += ur[nrr]/6
            else:
                usynatctic += ur[nrr]/9
        print(nsynatctic, nsemantic, usynatctic, usemantic)
        
        
        

D:\Library\Projects\amharic_word_embedding\data_handle.py:338: RuntimeWarning: invalid value encountered in true_divide
  return array / norms
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


7.931441649941231 7.449205851582461 2.2861349408470932 3.5049293241551083
7.701715948315732 6.978946020994152 2.122857560482329 4.278155295399345


In [4]:
seq_emb = parseVec('results/10_100-0.vec', ' ')

In [5]:
emb = seq_emb
nr = evaluate(word2int, normalize(emb), embed_size=emb.shape[1], corpus='data/10', analogy='data/ar_questions.txt')
ur = evaluate(word2int, emb, embed_size=emb.shape[1], corpus='data/10', analogy='data/ar_questions.txt')
nsemantic = 0
nsynatctic = 0
for nrr in nr:
    if 'total' in nrr:
        continue
    if "gram" in nrr:
        nsynatctic += nr[nrr]/6
    else:
        nsemantic += nr[nrr]/9

usemantic = 0
usynatctic = 0
for nrr in ur:
    if 'total' in nrr:
        continue
    if "gram" in nrr:
        usemantic += ur[nrr]/6
    else:
        usynatctic += ur[nrr]/9
print(nsynatctic, nsemantic, usynatctic, usemantic)

D:\Library\Projects\amharic_word_embedding\data_handle.py:338: RuntimeWarning: invalid value encountered in true_divide
  return array / norms
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


40.86916681017879 19.84990870412632 14.498638824971996 17.42845703447439


In [6]:
print(nr)

{'capital-common-countries': 5.9523809523809526, 'capital-world': 0.9979353062629044, 'currency': 0.0, 'city-in-state': 0.0, 'family': 8.823529411764707, 'gram1-adjective-to-adverb': 1.8421052631578947, 'gram2-opposite': 0.0, 'gram3-comparative': 4.273504273504273, 'gram4-superlative': 5.517241379310345, 'gram5-present-participle': 1.8279569892473118, 'gram6-nationality-adjective': 15.459723352318958, 'gram7-past-tense': 2.1839080459770117, 'gram8-plural': 1.075268817204301, 'gram9-plural-verbs': 8.68945868945869, 'total': 4.076063033717757}


In [10]:
a, b = 0,  0
for nrr in ur:
    if 'total' in nrr:
        continue
    if "gram" in nrr:
        a += ur[nrr]
    else:
        b += ur[nrr]


In [11]:
print(a/6, b/9)

2.904742839079065 1.3810883040648816


In [ ]:
all_vecs = np.zeros(shape=(58385, 100))
mmax = 11
result = np.zeros(shape=(mmax, 6))
for i in range(mmax):
    emb = parseVec('results/sq/w2v{0}.txt'.format(i), ' ')
    all_vecs += emb
    nr = evaluate(word2int, normalize(emb), embed_size=emb.shape[1])
    ur = evaluate(word2int, emb, embed_size=emb.shape[1])
    result[i][0] = nr['syntactic']
    result[i][1] = nr['semantic']
    result[i][2] = nr['pick-one-out']
    
    result[i][3] = ur['syntactic']
    result[i][4] = ur['semantic']
    result[i][5] = ur['pick-one-out']

In [ ]:
emb = all_vecs/mmax
print(evaluate(word2int, normalize(emb), embed_size=emb.shape[1]))
print(evaluate(word2int, emb, embed_size=emb.shape[1]))

In [ ]:
print(result.mean(axis=0))

In [ ]:
emb2D = np.zeros((seq_emb.shape[0], seq_emb.shape[1] * sem_emb.shape[1]))
for i in range(seq_emb.shape[0]):
    seq = seq_emb[i]
    sem = sem_emb[i]
    vec = np.outer(seq, sem).flatten()
    emb2D[i] = vec

In [ ]:
lines = open('data/newan2.txt', encoding='utf-8').readlines()
emb = emb2D
correct = 0
for line in lines:
    if ':' in line:
        continue
    try:
        line = [x.strip() for x in line[:-1].split(' ')]
        vs = [emb[word2int[w]] for w in line]
        v = vs[1] - vs[0] + vs[2]
        sims = emb.dot(v)
        sim_with_word = [(word, sims[word2int[word]])
                             for word in word2int.keys()]
        sorted_sim = sorted(sim_with_word, key=lambda t: t[1], reverse=True)[:4]
#         print(type(sorted_sim[0][1]), sorted_sim[0][1])
        result = [pred[0] for pred in sorted_sim]
        if not np.isnan(sorted_sim[0][1]):
#             print(result, line[-1])
            if line[-1] in result:
#                 print(result, line[-1])
                correct += 1
    except:
        pass
print(correct * 100.0 / (len(lines) - 2))

In [ ]:
# util = Utils(word2int, int2word, emb2D)
# lines = open('data/newan2.txt', encoding='utf-8').readlines()
# acc = np.array([0, 0])
# total = np.array([0, 0])
# ind = 0
# for line in lines[1:]:
#     if ':' in line[:-1]:
#         ind = 1
#         continue
#     ws = line[:-1].split()
#     result = util.solve(ws)
#     if result is None:
#         continue
#     if ws[-1] in result:
#         acc[ind] += 1
#     total[ind] += 1
# print(acc * 100 / total)

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# e = normalize(cew)
# # e = cew
# for line in lines:
# #     ws = "ልጅቱ ልጁ ወይዘሮ አቶ".split(' ')
#     if ":" in line:
#         continue
#     try:
#         ws = line[:-1].split(' ')
#         v = normalize(vec(ws[1], e) - vec(ws[0], e) + vec(ws[2], e))
#         V = np.dot(e, v)
#         sort = list(sorted(enumerate(V[1:]), key=lambda x: x[1], reverse=True))
#         so = [int2word[index[0]] for index in sort[:4]]
#         if ws[-1] in so:
#             print(so, ws[-1])
# #         break
#     except:
#         pass

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# words = []
# for line in lines:
#     ws = line[:-1].split(' ')
#     in_vocab = True
#     for w in ws:
#         if w not in word2int:
#             in_vocab = False
#         if in_vocab:
#             words.append(ws)
# ws = []
# for wl in words:
#     s = ' '.join(wl)
#     ws.append(s)
# ws = '\n'.join(ws)
# open('data/newan.txt', mode='w', encoding='utf-8').write(ws)

        
        

In [ ]:
# lines = open('data/analogies.txt', encoding='utf-8').readlines()
# words = []
# lns = {}
# for line in lines:
#     ws = line[:-1].split(' ')
#     in_vocab = True
#     for w in ws:
#         if w not in word2int:
#             in_vocab = False
#         if in_vocab:
#             ln = ' '.join(ws)
#             if ln not in lns:
#                 lns[ln] = len(lns)
#             ls = ' '.join(ws[2:4] + ws[0:2])
#             if ln not in lns:
#                 lns[ln] = len(lns)
# ws = '\n'.join(lns.keys())
# open('data/newan2.txt', mode='w', encoding='utf-8').write(ws)